# Webtoon Comments Scraping ! - All

In [ ]:
# import field

import re
import copy
import time
import sqlite3
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import random
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.options import Options

In [1]:
### 전 회차 url 따기
# 웹툰의 title
def get_all_episode_url(title_id):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    episode_urls = []
    for i in range(1, 1000):
        now_episode_urls = copy.deepcopy(episode_urls)
        url = f"https://comic.naver.com/webtoon/list?titleId={title_id}&page={i}&sort=ASC"
        driver.get(url)
        time.sleep(0.5)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        episode_divs = soup.find_all('li', {'class': 'EpisodeListList__item--M8zq4'})
        for episode_div in episode_divs:
            a_tag = episode_div.find('a')
            href = 'https://comic.naver.com' + a_tag['href']
            episode_urls.append(href)
        episode_urls = sorted(list(set(episode_urls)))
        if now_episode_urls == episode_urls:
            break
    driver.quit()
    return episode_urls

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import sqlite3

def handle_alert(driver):
    try:
        alert = Alert(driver)
        alert_text = alert.text
        print(f'경고창 닫힘: {alert_text}')
        alert.accept()
    except:
        pass

def put_comment_to_db(title_id, episode_url):
    # Chrome 드라이버 설정
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 백그라운드에서 실행
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    # 웹페이지 열기
    driver.get(episode_url)
    time.sleep(0.5)

    # 전체 댓글 클릭
    element = driver.find_element(By.XPATH, '/html/body/div[1]/div[5]/div/div/div[4]/div[1]/div[3]/div/div/div[4]/div[1]/div/ul/li[2]/a/span[2]')
    element.click()

    time.sleep(2)

    while True:
        try:
            element = driver.find_element(By.XPATH, '/html/body/div[1]/div[5]/div/div/div[4]/div[1]/div[3]/div/div/div[7]')
            element.click()
            time.sleep(0.5)  # 시간 지연을 늘림
            print('댓글 더보기 클릭됨!')
        except Exception as e:
            print('에러 발생:', e)
            handle_alert(driver)  # 경고창 처리
            break
        else:
            # /html/body/div[1]/div[5]/div/div/div[4]/div[1]/div[3]/div/div/div[7] 이 요소의 style의 display 속성이 'none'이 된다면 루프를 종료
            try:
                style = element.get_attribute('style')
            except Exception as e:
                print('댓글 너무 많은 에러..:', e)
                break
            else:
                if 'display: none' in style:
                    break

    # 댓글 요소 찾기
    comment_list_element = driver.find_element(By.XPATH, '/html/body/div[1]/div[5]/div/div/div[4]/div[1]/div[3]/div/div/div[6]/ul')

    # 모든 li 태그 찾기
    li_elements = comment_list_element.find_elements(By.TAG_NAME, 'li')

    # li 요소들의 텍스트를 리스트로 저장
    li_texts = []
    for li in li_elements:
        try:
            li_texts.append(li.get_attribute('innerHTML'))
        except Exception as e:
            print('li 텍스트를 가져오는 중 에러 발생:', e)
            handle_alert(driver)  # 경고창 처리

    # 해당 정보를 DB에 삽입
    conn = sqlite3.connect('webtoon_comments.db')
    cursor = conn.cursor()
    for li_text in li_texts:
        cursor.execute('''
            INSERT INTO comments (TitleID, URL, CommentLi) VALUES (?, ?, ?)
        ''', (title_id, episode_url, li_text))
        
    conn.commit()
    conn.close()

    # 드라이버 종료
    driver.quit()

In [4]:
# 만화는 격기 3반 !

"""
kk3b_id = '701535'
kk3b_urls = get_all_episode_url(kk3b_id)

for episode_url in kk3b_urls:
    put_comment_to_db(kk3b_id, episode_url)
"""
print()
# 댓글이 너무 많아서... 올릴 수가 X ...!